# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 2 2023 2:20PM,258577,10,SNAP32231,"Snap Medical Industries, LLC",Released
1,Mar 2 2023 2:16PM,258576,10,MSP219106,"Methapharm, Inc.",Released
2,Mar 2 2023 2:16PM,258576,10,MSP219107,"Methapharm, Inc.",Released
3,Mar 2 2023 2:06PM,258575,10,Enova-11447,Emerginnova,Released
4,Mar 2 2023 2:00PM,258573,16,9187446,Sartorius Bioprocess Solutions,Released
5,Mar 2 2023 1:58PM,258563,10,MTR-23-0292,"Senseonics, Incorporated",Released
6,Mar 2 2023 1:48PM,258572,19,GP03062023,"Granules Pharmaceuticals, Inc.",Released
7,Mar 2 2023 1:47PM,258571,19,ADV80013034,"AdvaGen Pharma, Ltd",Executing
8,Mar 2 2023 1:47PM,258571,19,ADV80013035,"AdvaGen Pharma, Ltd",Executing
9,Mar 2 2023 1:47PM,258571,19,ADV80013036,"AdvaGen Pharma, Ltd",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,258572,Released,1
34,258573,Released,1
35,258575,Released,1
36,258576,Released,2
37,258577,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258572,NaN,NaN,1.0
258573,NaN,NaN,1.0
258575,NaN,NaN,1.0
258576,NaN,NaN,2.0
258577,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258460,17.0,7.0,1.0
258461,12.0,15.0,6.0
258487,0.0,1.0,0.0
258493,0.0,0.0,2.0
258495,0.0,1.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258460,17,7,1
258461,12,15,6
258487,0,1,0
258493,0,0,2
258495,0,1,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258460,17,7,1
1,258461,12,15,6
2,258487,0,1,0
3,258493,0,0,2
4,258495,0,1,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258460,17,7,1
1,258461,12,15,6
2,258487,,1,
3,258493,,,2
4,258495,,1,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 2 2023 2:20PM,258577,10,"Snap Medical Industries, LLC"
1,Mar 2 2023 2:16PM,258576,10,"Methapharm, Inc."
3,Mar 2 2023 2:06PM,258575,10,Emerginnova
4,Mar 2 2023 2:00PM,258573,16,Sartorius Bioprocess Solutions
5,Mar 2 2023 1:58PM,258563,10,"Senseonics, Incorporated"
6,Mar 2 2023 1:48PM,258572,19,"Granules Pharmaceuticals, Inc."
7,Mar 2 2023 1:47PM,258571,19,"AdvaGen Pharma, Ltd"
11,Mar 2 2023 1:46PM,258570,10,"Nextsource Biotechnology, LLC"
13,Mar 2 2023 1:46PM,258569,10,Yusen – 3D Matrix
14,Mar 2 2023 1:46PM,258568,16,Capsugel


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 2 2023 2:20PM,258577,10,"Snap Medical Industries, LLC",,,1
1,Mar 2 2023 2:16PM,258576,10,"Methapharm, Inc.",,,2
2,Mar 2 2023 2:06PM,258575,10,Emerginnova,,,1
3,Mar 2 2023 2:00PM,258573,16,Sartorius Bioprocess Solutions,,,1
4,Mar 2 2023 1:58PM,258563,10,"Senseonics, Incorporated",,,1
5,Mar 2 2023 1:48PM,258572,19,"Granules Pharmaceuticals, Inc.",,,1
6,Mar 2 2023 1:47PM,258571,19,"AdvaGen Pharma, Ltd",,4,
7,Mar 2 2023 1:46PM,258570,10,"Nextsource Biotechnology, LLC",,,2
8,Mar 2 2023 1:46PM,258569,10,Yusen – 3D Matrix,,,1
9,Mar 2 2023 1:46PM,258568,16,Capsugel,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 2:20PM,258577,10,"Snap Medical Industries, LLC",1,,
1,Mar 2 2023 2:16PM,258576,10,"Methapharm, Inc.",2,,
2,Mar 2 2023 2:06PM,258575,10,Emerginnova,1,,
3,Mar 2 2023 2:00PM,258573,16,Sartorius Bioprocess Solutions,1,,
4,Mar 2 2023 1:58PM,258563,10,"Senseonics, Incorporated",1,,
5,Mar 2 2023 1:48PM,258572,19,"Granules Pharmaceuticals, Inc.",1,,
6,Mar 2 2023 1:47PM,258571,19,"AdvaGen Pharma, Ltd",,4,
7,Mar 2 2023 1:46PM,258570,10,"Nextsource Biotechnology, LLC",2,,
8,Mar 2 2023 1:46PM,258569,10,Yusen – 3D Matrix,1,,
9,Mar 2 2023 1:46PM,258568,16,Capsugel,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 2:20PM,258577,10,"Snap Medical Industries, LLC",1,,
1,Mar 2 2023 2:16PM,258576,10,"Methapharm, Inc.",2,,
2,Mar 2 2023 2:06PM,258575,10,Emerginnova,1,,
3,Mar 2 2023 2:00PM,258573,16,Sartorius Bioprocess Solutions,1,,
4,Mar 2 2023 1:58PM,258563,10,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 2:20PM,258577,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
1,Mar 2 2023 2:16PM,258576,10,"Methapharm, Inc.",2.0,NaN,NaN
2,Mar 2 2023 2:06PM,258575,10,Emerginnova,1.0,NaN,NaN
3,Mar 2 2023 2:00PM,258573,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
4,Mar 2 2023 1:58PM,258563,10,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 2:20PM,258577,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
1,Mar 2 2023 2:16PM,258576,10,"Methapharm, Inc.",2.0,0.0,0.0
2,Mar 2 2023 2:06PM,258575,10,Emerginnova,1.0,0.0,0.0
3,Mar 2 2023 2:00PM,258573,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
4,Mar 2 2023 1:58PM,258563,10,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3361198,19.0,7.0,0.0
12,258487,0.0,1.0,0.0
15,1551036,24.0,27.0,29.0
16,775676,3.0,0.0,0.0
19,1809783,25.0,4.0,0.0
22,258498,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3361198,19.0,7.0,0.0
1,12,258487,0.0,1.0,0.0
2,15,1551036,24.0,27.0,29.0
3,16,775676,3.0,0.0,0.0
4,19,1809783,25.0,4.0,0.0
5,22,258498,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,19.0,7.0,0.0
1,12,0.0,1.0,0.0
2,15,24.0,27.0,29.0
3,16,3.0,0.0,0.0
4,19,25.0,4.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,19.0
1,12,Released,0.0
2,15,Released,24.0
3,16,Released,3.0
4,19,Released,25.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,22
Status,,,,,,
Completed,0.0,0.0,29.0,0.0,0.0,0.0
Executing,7.0,1.0,27.0,0.0,4.0,0.0
Released,19.0,0.0,24.0,3.0,25.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,22
0,Completed,0.0,0.0,29.0,0.0,0.0,0.0
1,Executing,7.0,1.0,27.0,0.0,4.0,0.0
2,Released,19.0,0.0,24.0,3.0,25.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,22
0,Completed,0.0,0.0,29.0,0.0,0.0,0.0
1,Executing,7.0,1.0,27.0,0.0,4.0,0.0
2,Released,19.0,0.0,24.0,3.0,25.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()